In [68]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("labeled.csv", sep=",")
df['toxic'] = df['toxic'].apply(int)

In [3]:
df.shape

(14412, 2)

In [4]:
df['toxic'].value_counts()

0    9586
1    4826
Name: toxic, dtype: int64

In [5]:
df.query('toxic == 1')['comment']

0                     Верблюдов-то за что? Дебилы, бл...\n
1        Хохлы, это отдушина затюканого россиянина, мол...
2                                Собаке - собачья смерть\n
3        Страницу обнови, дебил. Это тоже не оскорблени...
4        тебя не убедил 6-страничный пдф в том, что Скр...
                               ...                        
14404    ЖЕНЩИНА это ВЕНЕЦ ТВОРЕНИЯ, помните это ваньки...
14405    Сейчас все авиакомпании мира вместе со специал...
14407    Вонючий совковый скот прибежал и ноет. А вот и...
14408    А кого любить? Гоблина тупорылого что-ли? Или ...
14410    КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...
Name: comment, Length: 4826, dtype: object

In [6]:
df.query('toxic == 0')['comment']

6        В шапке были ссылки на инфу по текущему фильму...
12       Почитайте посты у этого автора,может найдете ч...
17       Про графику было обидно) я так то проходил все...
28       https: pp.userapi.com c848520 v848520411 11627...
35       Возьмём как пример Россию, западноевропейские ...
                               ...                        
14392    ТАСС, 21 марта. Премьер-министр Новой Зеландии...
14399    65 А чего так мало? Вот у гражданина был рейти...
14406    Потому что запад прошел эту хуйню еще пару сот...
14409    Посмотрел Утомленных солнцем 2. И оказалось, ч...
14411    До сих пор пересматриваю его видео. Орамбо кст...
Name: comment, Length: 9586, dtype: object

In [7]:
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
# nltk.download('punkt')
# nltk.download('stopwords')

stemmer = SnowballStemmer(language='russian')
russian_stop_words = stopwords.words('russian')

def tokenize_sentence(sentence: str, remove_stop_words: bool=True):
    """Преобразует предложения в токены.
    По шагам:
    1) Токенизация - разбиение текста на отдельные слова и знаки пунктуации
    2) Удаляем знаки пунктуации
    3) Удаляем стоп-слова (часто используемые. не несущщие смысловой нагрузки)
    4) Стемминг (stem - корень) - удаление окончаний и приведение к нижнему регистру
    """
    tokens = word_tokenize(sentence, language='russian')
    tokens = [i for i in tokens if i not in string.punctuation]
    if remove_stop_words:
        tokens = [i for i in tokens if i not in russian_stop_words]
    tokens = [stemmer.stem(i) for i in tokens]
    return tokens

In [8]:
example = df.loc[1]['comment']
example

'Хохлы, это отдушина затюканого россиянина, мол, вон, а у хохлов еще хуже. Если бы хохлов не было, кисель их бы придумал.\n'

In [9]:
tokenize_sentence(example)

['хохл',
 'эт',
 'отдушин',
 'затюкан',
 'россиянин',
 'мол',
 'вон',
 'хохл',
 'хуж',
 'есл',
 'хохл',
 'кисел',
 'придума']

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x))

In [11]:
from sklearn.model_selection import train_test_split

features, labels = df['comment'], df['toxic']
features = vectorizer.fit_transform(features)
x_train, x_test, y_train, y_test = train_test_split(features, 
                                                   labels, 
                                                   test_size=0.2, 
                                                   random_state=27)

C:\Python310\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [12]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

knn = KNeighborsClassifier().fit(x_train, y_train)
tree = DecisionTreeClassifier().fit(x_train, y_train)
log_reg = LogisticRegression().fit(x_train, y_train)

In [52]:
from sklearn.metrics import recall_score, roc_auc_score

y_pred_knn = knn.predict(x_test)
y_pred_tree = tree.predict(x_test)
y_pred_log_reg = log_reg.predict(x_test)

knn_recall = recall_score(y_test, y_pred_knn)
tree_recall = recall_score(y_test, y_pred_tree)
log_reg_recall = recall_score(y_test, y_pred_log_reg)

knn_roc_auc = roc_auc_score(y_test, y_pred_knn)
tree_roc_auc = roc_auc_score(y_test, y_pred_tree)
log_reg_roc_auc = roc_auc_score(y_test, y_pred_log_reg)

print(f'knn_recall: {knn_recall}')
print(f'tree_recall: {tree_recall}')
print(f'log_regr_recall: {log_reg_recall}')
print('----------------------------------')
print(f'knn_roc_auc: {knn_roc_auc}')
print(f'tree_roc_auc: {tree_roc_auc}')
print(f'log_regr_roc_auc: {log_reg_roc_auc}')

knn_recall: 0.6415094339622641
tree_recall: 0.6205450733752621
log_regr_recall: 0.6048218029350105
----------------------------------
knn_roc_auc: 0.798463374316539
tree_roc_auc: 0.7524705667550236
log_regr_roc_auc: 0.79178363345299


In [61]:
from sklearn.model_selection import GridSearchCV


knn_grid = GridSearchCV(KNeighborsClassifier(), 
                        param_grid={'n_neighbors': range(1, 21)}, 
                        scoring='recall',
                        cv=3)
knn_grid.fit(x_train, y_train)

GridSearchCV(cv=3, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(1, 21)}, scoring='recall')

In [64]:
print(f'best params: {knn_grid.best_params_}')
print(f'best score: {knn_grid.best_score_}')

knn.set_params(**knn_grid.best_params_).fit(x_train, y_train)

y_pred_knn_best = knn.predict(x_test)
knn_recall_best = recall_score(y_test, y_pred_knn_best)
knn_roc_auc_best = roc_auc_score(y_test, y_pred_knn_best)
print(f'kNN recall with best params: {knn_recall_best}')
print(f'kNN roc_auc with best params: {knn_roc_auc_best}')
print('-------------------------------------------------')
print(classification_report(y_test, y_pred_knn_best))

best params: {'n_neighbors': 1}
best score: 0.6443707880236261
kNN recall with best params: 0.6519916142557652
kNN roc_auc with best params: 0.7448656878951194
-------------------------------------------------
              precision    recall  f1-score   support

           0       0.83      0.84      0.83      1929
           1       0.67      0.65      0.66       954

    accuracy                           0.78      2883
   macro avg       0.75      0.74      0.75      2883
weighted avg       0.78      0.78      0.78      2883



In [74]:
from sklearn.linear_model import LogisticRegressionCV

cs = np.logspace(-1, 3, 100)
log_reg_cv = LogisticRegressionCV(Cs=cs, 
                                  cv=3, 
                                  verbose=0, 
                                  max_iter=200,
                                  scoring='recall',
                                  random_state=27)
log_reg_cv.fit(x_train, y_train)

LogisticRegressionCV(Cs=array([1.00000000e-01, 1.09749877e-01, 1.20450354e-01, 1.32194115e-01,
       1.45082878e-01, 1.59228279e-01, 1.74752840e-01, 1.91791026e-01,
       2.10490414e-01, 2.31012970e-01, 2.53536449e-01, 2.78255940e-01,
       3.05385551e-01, 3.35160265e-01, 3.67837977e-01, 4.03701726e-01,
       4.43062146e-01, 4.86260158e-01, 5.33669923e-01, 5.85702082e-01,
       6.42807312e-01, 7.05...
       1.70735265e+02, 1.87381742e+02, 2.05651231e+02, 2.25701972e+02,
       2.47707636e+02, 2.71858824e+02, 2.98364724e+02, 3.27454916e+02,
       3.59381366e+02, 3.94420606e+02, 4.32876128e+02, 4.75081016e+02,
       5.21400829e+02, 5.72236766e+02, 6.28029144e+02, 6.89261210e+02,
       7.56463328e+02, 8.30217568e+02, 9.11162756e+02, 1.00000000e+03]),
                     cv=3, max_iter=200, random_state=27, scoring='recall')

In [75]:
print(f'best C: {log_reg_cv.C_}')

y_pred_lr_best = log_reg_cv.predict(x_test)
lr_recall_best = recall_score(y_test, y_pred_lr_best)
lr_roc_auc_best = roc_auc_score(y_test, y_pred_lr_best)
print(f'LR recall with best params: {lr_recall_best}')
print(f'LR roc_auc with best params: {lr_roc_auc_best}')
print('-------------------------------------------------')
print(classification_report(y_test, y_pred_lr_best))

best C: [572.23676594]
LR recall with best params: 0.7389937106918238
LR roc_auc with best params: 0.8404662695501628
-------------------------------------------------
              precision    recall  f1-score   support

           0       0.88      0.94      0.91      1929
           1       0.86      0.74      0.80       954

    accuracy                           0.87      2883
   macro avg       0.87      0.84      0.85      2883
weighted avg       0.87      0.87      0.87      2883

